In [1]:
import requests
import pandas as pd
from pathlib import Path
import time
import os



In [2]:
fund_info = pd.read_csv('fund_list.txt', delimiter=' ')

In [3]:
def get_nav_data(fund_name,fund_nencoding):
    headers = {
        "access-token": "b80e7f84dec465d910073c99aea0c4ffbc64c7587cb005b5dfe9cbd767028d74dcff01e9e093d23385a27848395b0551",
    }
    url = f"https://pyapi.huofuniu.com/pyapi/fund/viewv2?fid={fund_nencoding}&pt=1&shareToken="
    res = requests.get(url, headers=headers)
    nav_data = pd.DataFrame(res.json()["data"]["fund"]["prices"])
    nav_data = nav_data.rename(
        columns={
            "pd": "日期",
            "nav": "单位净值",
            "cnw": "累计净值",
            "cn": "复权净值",
            "pc": "涨跌幅",
            "drawdown": "最大回撤",
        }
    ).drop(columns=["e_pc"])
    nav_data = nav_data[["日期", "单位净值", "累计净值", "复权净值"]]
    nav_data["日期"] = pd.to_datetime(nav_data["日期"])
    nav_data.to_csv(
        Path(
            rf"data/{fund_name}_{nav_data["日期"].max().strftime("%Y-%m-%d")}.csv"
        ),
        index=False,
        encoding="utf-8-sig",
    )
    return nav_data

In [4]:
for row in fund_info.itertuples(index=False, name=None):
    files_list_series = pd.Series([i for i in Path("./data").rglob("*.csv")])
    contains_specific_string = files_list_series.apply(lambda x: row[0] in str(x.name)).any()
    if contains_specific_string:
        files_to_delete = files_list_series[files_list_series.apply(lambda x: row[0] in str(x.name))]
        for file_path in files_to_delete:
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
    get_nav_data(row[0],row[1])
    print(f"正在爬取{row[0]}净值")
    time.sleep(2)
print("爬取完成")

Deleted file: data\济海实恒对冲二号_2025-01-21.csv
正在爬取济海实恒对冲二号净值
Deleted file: data\草本优益1号_2025-01-17.csv
正在爬取草本优益1号净值
Deleted file: data\草本致远1号_2025-01-17.csv
正在爬取草本致远1号净值
Deleted file: data\星恒哲萌1号_2025-01-17.csv
正在爬取星恒哲萌1号净值
Deleted file: data\均成CTA1号_2025-01-17.csv
正在爬取均成CTA1号净值
Deleted file: data\旭诺CTA三号_2025-01-17.csv
正在爬取旭诺CTA三号净值
Deleted file: data\宏锡量化CTA7号_2025-01-17.csv
正在爬取宏锡量化CTA7号净值
Deleted file: data\千衍九凌1号_2025-01-17.csv
正在爬取千衍九凌1号净值
Deleted file: data\众壹资产量合兴成一号_2025-01-17.csv
正在爬取众壹资产量合兴成一号净值
Deleted file: data\弘源多元化CTA_2025-01-17.csv
正在爬取弘源多元化CTA净值
Deleted file: data\会世元丰CTA1号_2025-01-17.csv
正在爬取会世元丰CTA1号净值
Deleted file: data\远澜云杉_2025-01-17.csv
正在爬取远澜云杉净值
Deleted file: data\量派CTA七号_2025-01-17.csv
正在爬取量派CTA七号净值
Deleted file: data\安贤CTA量化一号_2025-01-17.csv
正在爬取安贤CTA量化一号净值
Deleted file: data\远澜翠柏1号_2024-12-20.csv
正在爬取远澜翠柏1号净值
Deleted file: data\旌安1号_2024-07-26.csv
正在爬取旌安1号净值
Deleted file: data\辰元优选稳健_2025-01-17.csv
正在爬取辰元优选稳健净值
Deleted file: data\殊馥馥源套利1号_2025-01-17.csv
正在爬取殊馥馥